In [1]:
#модуль для считывания данных с онлайн ресурсов/различных компаний
from pandas_datareader import data
import datetime
from bokeh.plotting import figure, show, output_file

# модуль компонентов из интерфейса встраивания Bokeh
from bokeh.embed import components

#модуль сетевой доставки контента
from bokeh.resources import CDN

start_data = datetime.datetime(2018,9,1)
end_data = datetime.datetime(2018,9,24)

#в параметры передаём имя компании(в данном случае запасное имя Google), источник данных, 
#промежуток времени, из которого получаем данные; тип dataframe
df = data.DataReader(name='GOOG', data_source='yahoo', start=start_data, end=end_data)

#создаём новый столбец в датафрейме, обозначающий статус цены(увеличивается или уменьшается)
#значение в каждой ячейке будет устанавливаться в зависимости от цен на открытие или закрытие в этот день

def inc_dec(cl, op):
    if cl > op:
        value = 'Increase'
    elif cl < op:
        value = 'Decrease'
    else:
        value = 'Equal'
    return value

#устанавливаем статус с помощью функции, перебирая в цикле все дни со значениями close open попарно
df['Status'] = [inc_dec(cl,op) for cl, op in zip(df.Close, df.Open)]

#создаём столбец с координатами серидины прямоугольника по Y
df['Middle'] = (df.Close + df.Open)/2

#столбец для высоты
df['Height'] = abs(df.Close - df.Open)

p = figure(x_axis_type='datetime', width=1500, height=800)

# график масштабируется вместе с окном(вместо устаревшего responsive = True)
p.sizing_mode = "scale_both"

#заголовок диаграммы
p.title.text = ('Candlestick Chart')

#устанавливаем прозрачность сетки(от 0 до 1)
p.grid.grid_line_alpha = 0.3

#переводим 12 часов в миллисекунды
hours_12 = 12 * 60 * 60 * 1000

#используем сегменты для отображения максимальной и минимальной цены за день; 
#4 параметра: x,y координаты верхней точки и x,y координаты нижней точки
#отрисовываем сегменты до прямоугольников, так как в Bokeh новые слои накладываются поверх старых

p.segment(df.index, df.High, df.index, df.Low, color = 'black')


#определем координаты центров прямоугольника; координаты по оси X - даты, когда в столбце Status значение Increase 
#(цена закрытия торгов выше цены открытия)
#строки индексируются по дате, поэтому тспользуем index
#координаты по оси Y - сумма цен открытия и закрытия попалам, вычисляется в столбце Middle; 
#ширина прямоугольников будет 12 часов(откладывается + и - 6 часов от центра)
#высота - разница между ценой закрытия и открытия, столбец Height
#указываем цвет прямоугольников и цвет границ 

p.rect(df.index[df.Status == 'Increase'],df.Middle[df.Status == 'Increase'], 
       hours_12, df.Height[df.Status == 'Increase'], fill_color = 'green', line_color = 'black')

#координаты по оси X - даты, когда в столбце Status значение Decrease
#(цена закрытия торгов ниже цены открытия), цвет прямоугольников красный

p.rect(df.index[df.Status == 'Decrease'],df.Middle[df.Status == 'Decrease'], 
       hours_12, df.Height[df.Status == 'Decrease'], fill_color = 'red', line_color = 'black')

#для встраивания графика Bokeh в Flask необходимо 4 вещи: 
#исходный код диаграммы JavaScript, кода диаграммы HTML, динамическая ссылка на файл js и динамическая ссылка на файл css

#отображает картеж из 2х элементов: исходного кода JavaScript, кода HTML с div;
#передаём их в функцию компонентов
#оба элемента типа str; передаём их в tamplates (Section10)
script, division1 = components(p)

#список ссылок на js файлы
cdn_js = CDN.js_files

#список ссылок на css файлы
cdn_css = CDN.css_files


# отображение графика в локальном html файле, сохраняется на компе
output_file('CS.html')

show(p)


In [96]:
#получаем первый элемент списка - динамическую ссылку на js файл(при обновлении версии Bokeh, ссылка будет тоже обновляться) 
cdn_js[0]

'https://cdn.pydata.org/bokeh/release/bokeh-0.13.0.min.js'

In [97]:
#получаем динамическую ссылку на css файл
cdn_css[0]

'https://cdn.pydata.org/bokeh/release/bokeh-0.13.0.min.css'